# Reading Reddit data

![png](https://upload.wikimedia.org/wikipedia/en/thumb/b/b4/Reddit_logo.svg/1280px-Reddit_logo.svg.png)

It's possible load data from Reddit throw its url using '.json' parameter

In [3]:
!curl -s http://www.reddit.com/r/technology/.json | head -n 1

{"kind": "Listing", "data": {"modhash": "", "children": [{"kind": "t3", "data": {"domain": "self.technology", "banned_by": null, "media_embed": {}, "subreddit": "technology", "selftext_html": "&lt;!-- SC_OFF --&gt;&lt;div class=\"md\"&gt;&lt;h2&gt;Greetings Good People of &lt;a href=\"/r/Technology\"&gt;/r/Technology&lt;/a&gt;,&lt;/h2&gt;\n\n&lt;p&gt;Welcome to the &lt;a href=\"/r/Technology\"&gt;/r/Technology&lt;/a&gt; Tech Support / General Discussion Thread.&lt;/p&gt;\n\n&lt;p&gt;&lt;strong&gt;All questions must be submitted as top comments&lt;/strong&gt; (direct replies to this post).&lt;/p&gt;\n\n&lt;p&gt;As always, we ask that you keep it civil, abide by the &lt;a href=\"http://www.reddit.com/rules\"&gt;rules of reddit&lt;/a&gt; and mind your &lt;a href=\"https://www.reddit.com/wiki/reddiquette\"&gt;reddiquette&lt;/a&gt;.  Please hit the report button on any activity that you feel may be in violation of any of the guidelines listed above.&lt;/p&gt;\n\n&lt;p&gt;Should you require 

Let's save the data into a file ...

In [14]:
!curl -s http://www.reddit.com/r/technology/.json | python -m json.tool > ../data/reddit.json

# Loading into MongoDB

In [15]:
import json
import urllib2
import pymongo

# connect to mongo
connection = pymongo.MongoClient("mongodb://localhost")

# get a handle to the reddit database
db=connection.reddit
stories = db.stories

# drop existing collection
stories.drop()

# get the reddit home page
reddit_page = urllib2.urlopen("http://www.reddit.com/r/technology/.json")

# parse the json into python objects
parsed = json.loads(reddit_page.read())

# iterate through every news item on the page
for item in parsed['data']['children']:
    # put it in mongo
    stories.insert_one(item['data'])

# Finding with RegEx

In [17]:
query = {'title':{'$regex':'apple|google', '$options':'i'}}
projection = {'title':1, '_id':0}

try:
    cursor = stories.find(query, projection)

except Exception as e:
    print "Unexpected error:", type(e), e

for doc in cursor:
    print doc

{u'title': u'DOJ: Apple owns your iPhone\u2019s software, so it should have a backdoor'}
{u'title': u"Apple's EULA Gives It License to Invade Your Privacy, Government Claims"}
{u'title': u'Apple Hires Nvidia Artificial Intelligence Director Jonathan Cohen'}
